In [32]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.metrics import MeanSquaredError

In [1]:
df = pd.read_csv('my_data.csv',dtype='float')
df

In [2]:
# df = pd.read_csv('new_IV1_W_L_1Id_expanded.csv',dtype='float')
df = df.sample(frac=1).reset_index(drop=True)

df['id'] = df['id'].apply(lambda x: np.log(1/x))



df.head()



In [36]:
maxvals=np.array(df.max())
minvals=np.array(df.min())

In [37]:
df_array=df.iloc[:,0:].values
u=4

df_array_std=(df_array-minvals)/(maxvals-minvals)
df_array_scaled = df_array_std * (1 - 0) + 0
xscale=df_array_scaled[:,0:u]

yscale=df_array_scaled[:,u:]

# yscale1=df_array_scaled[:,3:13]
# yscale2=df_array_scaled[:,13:22]

# yscale3=df_array_scaled[:,22:28]
# yscale3=df_array_scaled[:,28:]
# yscale1.shape

In [39]:
X_train, X_test, Y_train, Y_test = train_test_split(xscale, yscale,test_size=0.5, train_size=0.5, random_state=42)

In [41]:
aa='relu'
model2 = Sequential()
model2.add(Dense(300, input_dim=u, kernel_initializer='normal', activation=aa))
# model2.add(Dense(16, activation=aa))
# model2.add(Dense(64, activation=aa))

# model2.add(Dense(64, activation=aa))
# model2.add(Dense(128, activation=aa))
# model2.add(Dense(64, activation=aa))



model2.add(Dense(3, activation='linear'))
model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 300)               1500      
                                                                 
 dense_12 (Dense)            (None, 3)                 903       
                                                                 
Total params: 2,403
Trainable params: 2,403
Non-trainable params: 0
_________________________________________________________________


In [43]:
# model1.compile(loss='mae', optimizer='Adam', metrics=[MeanSquaredError()])
# model1.optimizer.lr = 0.001
model2.compile(loss=mse, optimizer='Adamax', metrics=[MeanSquaredError()])
model2.optimizer.lr = 0.0006
# model3.compile(loss='mae', optimizer='Adam', metrics='accuracy')
# model3.optimizer.lr = 0.0006
# model4.compile(loss='mae', optimizer='Adam', metrics='accuracy')
# model4.optimizer.lr = 0.001

history1 = model1.fit(X_train, Y_train, epochs=1000, batch_size=32, verbose=0)

In [44]:
history2 = model2.fit(X_train, Y_train, epochs=500, batch_size=64, verbose=0)

In [49]:
def model_details(model):
    # Print model summary
    model.summary()

    # Print input shape
    # print(f'Input shape: {model.layers[0].input_shape[1]}')


    # Print output shape
    # print(f'Output shape: {model.layers[-1].output_shape[1]}')

    # Print number of layers
    # print(f'Number of layers: {len(model.layers)}')



    # Print activation function used
    # for layer in model.layers:
        # print(f'Activation function of layer {layer.name}: {layer.activation.__name__}')

    N_inputs=model.layers[0].input_shape[1]
    N_outputs=model.layers[-1].output_shape[1]
    N_layers=len(model.layers)
    N_neurons=np.zeros(N_layers,dtype=int)
    # Number of nodes in each layer:
    for i in range(N_layers):
      N_neurons[i]=int(model.layers[i].output_shape[1])
    # print('N_neurons', N_neurons)
    Activation=[]
    for layer in model.layers:
      Activation.append(layer.activation.__name__)

    # print('Activation',Activation)
    # Print weights of each layer


    weights = {}
    biases = {}

    for i, layer in enumerate(model.layers):
      for j in range(int(N_neurons[i])):
          weights["Weights_layer{}_neuron{}".format(i,j)]=layer.get_weights()[0][:,j]
      biases["Biases_layer{}".format(i)]=layer.get_weights()[1]
    # print(weights)
    # print(biases)
    return(N_inputs,N_layers,N_neurons,weights,biases,Activation)



def verilog_code(N_inputs,N_layers,N_neurons,weights,biases,Activation,X_test):
    N_inputs=N_inputs
    N_layers=N_layers
    N_neurons=N_neurons
    weights=weights
    biases=biases
    activation=Activation
    X=X_test
    Z_layer=0
    for i in range(N_layers):
        # print('layer',i)
        Zneuron=0
        Xnext=np.zeros(int(N_neurons[i]))
        if i ==0:

            inputs_len=N_inputs

        else:
            inputs_len=int(N_neurons[i-1])
            X=Xnow
        for j in range(int(N_neurons[i])):
            Zip=0
            # print('neuron',j)
            # print('len_inputs',inputs_len)
            # print('X',X)
            for k in range(inputs_len):
                # print('input',k)
                # print('weight*',weights["Weights_layer{}_neuron{}".format(i,j)][k],'input',X[k])

                Zip  += weights["Weights_layer{}_neuron{}".format(i,j)][k]*X[k]
                # print('Zip',Zip)

            Zneuron = Zip + biases["Biases_layer{}".format(i)][j]
            # print('Zneuron',Zneuron)

            if activation[i]== 'relu':
            
                Xnext[j]=max(0,Zneuron)
            elif activation[i]== "sigmoid":
                Xnext[j]=1/(1+np.exp(-Zneuron))
            elif activation[i]== 'linear':
                Xnext[j]=Zneuron
            # print('Xnext',Xnext)
        Xnow=Xnext

    return(Xnow)


    




# Creating a model
model = model2

# Display model details
N_inputs,N_layers,N_neurons,weights,biases,Activation=model_details(model)

X_now=verilog_code(N_inputs,N_layers,N_neurons,weights,biases,Activation,X_test)
print('X_now',X_now)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 300)               1500      
                                                                 
 dense_12 (Dense)            (None, 3)                 903       
                                                                 
Total params: 2,403
Trainable params: 2,403
Non-trainable params: 0
_________________________________________________________________
X_now [0.89654595 0.89654794 0.89667548]


In [50]:
model2.predict(xscale_test[0:2], verbose=0)

array([[0.896546  , 0.8965479 , 0.89667547],
       [0.8945883 , 0.8945923 , 0.89474154]], dtype=float32)

In [ ]:
minvals

array([3.33333333e+01, 2.50000000e+00, 1.00000000e-02, 0.00000000e+00,
       8.59145537e+00])

In [ ]:
maxvals

array([83.33333333, 14.16666667,  0.7       ,  0.71      , 26.69080158])

In [ ]:
# Define the Verilog-A code as a string
header = """
// Verilog-A code example


`include "constants.vams"
`include "disciplines.vams"


module NN_to_Va(g,s,d);
    inout g,s,d;
    electrical g,s,d;
  
    parameter real L = 0.012   from [0.01:0.1];	 //um
    parameter real W = 0.02   from [0.01:0.4];	 //um
    real Vgs, Vds, Weff, Li;
    real Xin[0:3];
    real Zip, Zneuron, input_length; 

    integer i, j, k;

"""
model_parameters = """
    real N_inputs={};
    real N_layers={};
  
"""

scaling_parameters = """
    real min1 = {};
    real min2 = {};
    real min3 = {};
    real min4 = {};
    real max1 = {};
    real max2 = {};
    real max3 = {};
    real max4 = {};
    real minid = {};
    real maxid = {};
  
"""
# Write the Verilog-A code to a file
with open('verilog_a_file_id.va', 'w') as file:
    file.write(header)
    file.write(model_parameters.format(N_inputs,N_layers))
    file.write(scaling_parameters.format(minvals[0], minvals[1], minvals[2], minvals[3], maxvals[0], maxvals[1], maxvals[2], maxvals[3], minvals[4], maxvals[4]))

    file.write("    real N_neurons[0:{}]=".format(N_layers-1))
    file.write('{')
    file.write('{}'.format(N_neurons[0]))
    for i in N_neurons[1:]:
      file.write(',{}'.format(i))
    file.write('};\n')


    for i in range(N_layers):
      file.write("    real Y_layer_{}[0:{}];\n".format(i,int(N_neurons[i]-1)))
      for j in range(int(N_neurons[i])):
        file.write("    real weights_layer_{}_neuron_{}[0:{}]=".format(i,j,weights["Weights_layer{}_neuron{}".format(i,j)].shape[0]-1))
        file.write('{')
        file.write('{}'.format(weights["Weights_layer{}_neuron{}".format(i,j)][0]))
        for z in weights["Weights_layer{}_neuron{}".format(i,j)][1:]:
            file.write(',{}'.format(z))
        file.write('};\n')
        file.write("    real bias_layer_{}_neuron_{}={};\n".format(i,j,biases["Biases_layer{}".format(i)][j]))
    file.write("    analog begin\n")
    file.write("        Vgs=V(g)-V(s);\n")
    file.write("        Vds=V(d)-V(s);\n")
    file.write("        Li=1/L;\n")
    file.write("        Weff=2*W+0.01;\n")
    for i in range(N_layers):
      for j in range(int(N_neurons[i])):
        if i == 0:
            file.write("        input_length = {};\n        Zip=0;\n".format(N_inputs))
            file.write("        Xin[0]=(Li-min1)/(max1-min1);\n")
            file.write("        Xin[1]=((Weff/L)-min2)/(max2-min2);\n")
            file.write("        Xin[2]=(Vds-min3)/(max3-min3);\n")
            file.write("        Xin[3]=(Vgs-min4)/(max4-min4);\n")
            file.write("        for (k = 0; k < input_length; k = k + 1) begin\n")
            file.write("            Zip = Zip + weights_layer_{}_neuron_{}[k]*Xin[k];\n".format(i,j))
        if i>0:
            # file.write("        X=Y_layer_{};\n".format(i-1))
            file.write("        input_length = {};\n        Zip=0;\n".format(int(N_neurons[i-1])))

            file.write("        for (k = 0; k < input_length; k = k + 1) begin\n")

            file.write("            Zip = Zip + weights_layer_{}_neuron_{}[k]*Y_layer_{}[k];\n".format(i,j,i-1))
        file.write("        end\n")
        file.write("        Zneuron = Zip + bias_layer_{}_neuron_{};\n".format(i,j))

        if Activation[i]== "relu":
 
            file.write("        Y_layer_{}[{}]=max(0,Zneuron);\n".format(i,j))
        if Activation[i]== "sigmoid":
            file.write("        Y_layer_{}[{}]=1/(1+exp(-1*Zneuron));\n".format(i,j))
                
        if Activation[i]== "linear":
            file.write("        Y_layer_{}[{}]=Zneuron;\n".format(i,j))

    file.write("    I(d,s)<+(1/(exp((Y_layer_{}[0])*(maxid-minid)+minid)));\n".format(i))
    file.write("  end\n")
    file.write("endmodule\n")

print("Verilog-A file generated successfully.")





Verilog-A file generated successfully.
